In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
sys.path.append('../../src/dsets/gaussian_mixture')
from dset import get_dataloaders
sys.path.append('../../src/vae')
from model import init_specific_model
from losses import Loss
from training import Trainer
from utils import plot_2d_samples, plot_2d_latent_samples, plot_traversals
import pickle as pkl
import pandas as pd
import seaborn as sns
from sim_gaussian_mixture import p, define_dataloaders

In [2]:
dirs = ["vary_lamPT_mu_warmstart_seed=13"]
results = []
models = []
for i in range(len(dirs)):
    # load results
    out_dir = opj("/home/ubuntu/local-vae/notebooks/ex_gaussian_mixture/results", dirs[i])
    fnames = sorted(os.listdir(out_dir))

    results_list = []
    models_list = []
    for fname in fnames:
        if fname[-3:] == 'pkl':
            results_list.append(pkl.load(open(opj(out_dir, fname), 'rb')))
        if fname[-3:] == 'pth':
            model = init_specific_model(orig_dim=results_list[-1]['orig_dim'], 
                                        latent_dim=results_list[-1]['latent_dim'], 
                                        hidden_dim=results_list[-1]['hidden_dim']).to(device)
            model.load_state_dict(torch.load(opj(out_dir, fname)))
            models_list.append(model)
            
    results.append(pd.DataFrame(results_list))
    models.append(models_list)

## Plotting results

In [ ]:
ax = sns.pointplot(x="lamPT", y="disentanglement_metric", hue=None, data=results[0])

In [ ]:
ax = sns.pointplot(x="lamPT", y="disentanglement_metric", hue="mu", data=results[0])

In [ ]:
ax = sns.pointplot(x="lamPT", y="reconstruction_loss", hue="mu", data=results[0])

In [ ]:
ax = sns.pointplot(x="lamPT", y="kl_normal_loss", hue="mu", data=results[0])

In [ ]:
ax = sns.pointplot(x="lamPT", y="total_correlation", hue="mu", data=results[0])

In [ ]:
ax = sns.pointplot(x="lamPT", y="pt_local_independence_loss", hue="mu", data=results[0])

## Visualization

In [ ]:
# GET MODEL AND RESULT
idx = 200
model = models[0][idx]
result = results[0].loc[idx]
print('disentanglement metric: {:.3f}'.format(result['disentanglement_metric']))

params_to_vary = ['num_epochs', 'seed', 'hidden_dim', 'eps', 'beta', 'mu', 'lamPT', 'lamCI', 'dirname', 'latent_means', 'latent_vars']
for name in params_to_vary:
    setattr(p, name, result[name])
    
# SEED
random.seed(p.seed)
np.random.seed(p.seed)
torch.manual_seed(p.seed)    

# GET DATALOADERS
(_, _), (test_loader, test_latents) = define_dataloaders(p)

# EVALUATE TEST DATA
data = test_loader.dataset.data.to(device)
recon_data, latent_dist, latent_sample = model(data)
plot_2d_latent_samples(latent_sample)
plt.title("Plot of test latent samples")
plt.show()

In [ ]:
plot_traversals(model, data, lb=1000, ub=2000, num=100, max_traversal=.02)
plt.title("Plot of traversals")
plt.show()

In [ ]:
plot_traversals(model, data, lb=0, ub=5000, num=20,
                draw_data=True, draw_recon=True, max_traversal=.1)